In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, SubsetRandomSampler


def img_size(img_w,filter_size,padding,stride):
        return (1/2)*(1+(img_w-filter_size+(2*padding))/stride)

# Define the ConvNet class
class ConvNet(nn.Module):
    def __init__(self, in_channels=3, num_filters=[32,64,128,256,512], filter_size=[3,5,5,7,7], activation=nn.ReLU(), stride=1, padding=1, pool_size=(2,2), fc_size=128, num_classes=10):
        super(ConvNet, self).__init__()
        self.channels = in_channels
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.activation = activation
        self.stride = stride
        self.padding = padding
        self.pool_size = pool_size
        self.fc_size = fc_size
        self.num_classes = num_classes

        # Define convolutional layers
        self.conv1 = nn.Conv2d(self.channels, self.num_filters[0], self.filter_size[0], stride=self.stride, padding=self.padding)
        self.conv2 = nn.Conv2d(self.num_filters[0], self.num_filters[1], self.filter_size[1], stride=self.stride, padding=self.padding)
        self.conv3 = nn.Conv2d(self.num_filters[1], self.num_filters[2], self.filter_size[2], stride=self.stride, padding=self.padding)
        self.conv4 = nn.Conv2d(self.num_filters[2], self.num_filters[3], self.filter_size[3], stride=self.stride, padding=self.padding)
        self.conv5 = nn.Conv2d(self.num_filters[3], self.num_filters[4], self.filter_size[4], stride=self.stride, padding=self.padding)

        # Define activation function
        self.activation = activation

        # Define max pooling layers
        self.pool = nn.MaxPool2d(self.pool_size,stride = 2)

        nxt_size1 = img_size(224,self.filter_size[0],self.padding,self.stride)
        nxt_size2 = img_size(nxt_size1,self.filter_size[1],self.padding,self.stride)
        nxt_size3 = img_size(nxt_size2,self.filter_size[2],self.padding,self.stride)
        nxt_size4 = img_size(nxt_size3,self.filter_size[3],self.padding,self.stride)
        nxt_size5 = img_size(nxt_size4,self.filter_size[4],self.padding,self.stride)
        print(nxt_size5)
        nxt_size5 = int(nxt_size5)

        # Define fully connected layer
        self.fc = nn.Linear(self.num_filters[4]*(nxt_size5)**2, self.fc_size)  # Assuming input size of 32x32 after 5 max-pooling layers

        # Output layer
        self.output_layer = nn.Linear(self.fc_size, self.num_classes)




    def forward(self, x):
        # Convolutional layers
        x = self.conv1(x)
        x = self.activation(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.activation(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = self.activation(x)
        x = self.pool(x)

        x = self.conv4(x)
        x = self.activation(x)
        x = self.pool(x)

        x = self.conv5(x)
        x = self.activation(x)
        x = self.pool(x)

        # Flatten the output for the fully connected layer
        x = x.view(x.size(0), -1)

        # Fully connected layer
        x = self.fc(x)
        x = self.activation(x)

        # Output layer
        x = self.output_layer(x)

        return x

# Create an instance of the ConvNet model
model = ConvNet()
print(model)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


print('Training complete!')


3.625
ConvNet(
  (activation): ReLU()
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(256, 512, kernel_size=(7, 7), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=4608, out_features=128, bias=True)
  (output_layer): Linear(in_features=128, out_features=10, bias=True)
)
Training complete!
